In [1]:
import numpy as np
import tensorflow as tf
import pickle

from keras.models import load_model
from keras.layers import TextVectorization

model = load_model('final.h5')
from_disk = pickle.load(open("vectorconfig.pkl", "rb"))
vectoriser = TextVectorization.from_config(from_disk['config'])

vectoriser.adapt(tf.data.Dataset.from_tensor_slices(["xyz"])) # call adapt on dummy data
vectoriser.set_weights(from_disk['weights'])

from_disk = pickle.load(open("storeconfig.pkl", "rb"))
store_vectoriser = TextVectorization.from_config(from_disk['config'])

store_vectoriser.adapt(tf.data.Dataset.from_tensor_slices(["xyz"])) # call adapt on dummy data
store_vectoriser.set_weights(from_disk['weights'])

encoder = pickle.load(open("encoder.pickle", "rb"))

In [3]:
encoder.classes_

array(['alcohol', 'cleaning & laundry', 'clothes & accesories',
       'clothes & accessories', 'drinks', 'eating out', 'electrical',
       'entertainment', 'gardening', 'groceries', 'health & pharmacy',
       'homeware', 'nicotine', 'other', 'personal hygiene', 'snacks'],
      dtype=object)

In [24]:
items = [
    ['fried chicken'],
    ['poems'],
    ['biscuits'],
    ['ice cream'],
    ['conditioner'],
    ['lemon tea'],
    ['lemon iced tea'],
    ['lemon ice tea'],
    ['fried chicken'],
    ['poems'],
    ['biscuits'],
    ['ice cream'],
    ['vodka'],
    ['lemon tea'],
    ['lemon iced tea'],
    ['lemon ice tea']
    ]
    
stores = [
    ['t4'],
    ['waterstones'],
    ['tesco'],
    ['tesco'],
    ['sainsburys'],
    ['aldi'],
    ['lidl'],
    ['t4'],
    ['tesco'],
    ['tesco'],
    ['sainsburys'],
    ['asda'],
    ['tesco'],
    ['tesco'],
    ['tesco'],
    ['tesco']
    ]

In [27]:
product_string = tf.keras.Input(shape=(1,), dtype="string")
store_string = tf.keras.Input(shape=(1,), dtype="string")

x = vectoriser(product_string)
x2 = store_vectoriser(store_string)

class_names= encoder.classes_

predictions = model([x, x2])


e2e_model = tf.keras.Model([product_string, store_string], predictions)

products_input = tf.convert_to_tensor(items)
store_input = tf.convert_to_tensor(stores)


probabilities = e2e_model.predict([products_input, store_input])

for i in tf.range(len(probabilities)):
    if float('{0:.2f}'.format(np.max(probabilities[i]))) <0.5:
        print("\nBELOW 0.5 CONF")
        print(""+products_input.numpy()[i][0].decode('UTF-8') + ", " + store_input.numpy()[i][0].decode('UTF-8'))
        print("(" + class_names[np.argmax(probabilities[i])] + ": " + str(float('{0:.3f}'.format(np.max(probabilities[i])))) + ")\n")
    else:
        print(products_input.numpy()[i][0].decode('UTF-8') + ", " + store_input.numpy()[i][0].decode('UTF-8') + "\n(" + class_names[np.argmax(probabilities[i])] + ": " + str(float('{0:.3f}'.format(np.max(probabilities[i])))) + " conf)")

fried chicken, t4
(eating out: 0.724 conf)

BELOW 0.5 CONF
poems, waterstones
(eating out: 0.257)

biscuits, tesco
(groceries: 0.591 conf)
ice cream, tesco
(snacks: 1.0 conf)

BELOW 0.5 CONF
conditioner, sainsburys
(personal hygiene: 0.401)

lemon tea, aldi
(drinks: 0.889 conf)
lemon iced tea, lidl
(drinks: 0.744 conf)
lemon ice tea, t4
(drinks: 0.556 conf)
fried chicken, tesco
(groceries: 0.843 conf)

BELOW 0.5 CONF
poems, tesco
(groceries: 0.333)


BELOW 0.5 CONF
biscuits, sainsburys
(groceries: 0.375)

ice cream, asda
(snacks: 0.998 conf)

BELOW 0.5 CONF
vodka, tesco
(groceries: 0.333)

lemon tea, tesco
(drinks: 0.838 conf)
lemon iced tea, tesco
(drinks: 0.635 conf)
lemon ice tea, tesco
(drinks: 0.579 conf)
